<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学術分野の分類（複数）
## kakenhi_categorize_multi.ipynb
kakenhi_fine_tuning.ipynbでファインチューニングにより作成したモデルを用いて、新たな複数の学術文書（実際には科研費の概要テキスト）を11の大区分に分類するプログラム。  

### ライブラリのインストール
SageMakerでは初回だけで良い（Google colaboratoryでは毎回）  
ライブラリTransformers、およびnlpをインストールします。  

In [ ]:
# pipのアップデート（たまに走らせても良いかも）
!pip list
!python -m pip install --upgrade pip

In [ ]:
# ライブラリのインストール
!pip install torch
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic

### Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
# SageMakerでは不要
# from google.colab import drive
# drive.mount("/content/drive/")

# 以下途中まで不要

## ファインチューニング用データの読み込み＆書き出し
### 科研費データの読み込み、整理
科研費データベースからダウンロードしたcsvファイルを直接読む  
必要なデータを取り出す。

In [ ]:


import pandas as pd

# 科研費データベースからダウンロードした未加工のcsvファイルを指定
# 基盤BC　2019年度～2021年度の４年分（2018年度は”研究開始時の研究の概要”が無い。2022年度は後半の検証に用いる）
open_original_csv = "KibanBC_2019-2021.csv"

# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "../data/" # SageMaker 

# csvファイルを開く
raw_data = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()

# 今後必要な行だけを取り出し、リネーム
# kadai = raw_data[["研究課題/領域番号", "審査区分", "研究課題名", "研究開始時の研究の概要", "研究成果の概要", "研究実績の概要", "キーワード"]]
# kadai.columns = ["ID", "ShoKubun", "Title", "Abst1", "Abst2", "Abst3", "Keyword"]
kadai = raw_data[["研究課題/領域番号", "審査区分", "研究開始時の研究の概要"]]
kadai.columns = ["ID", "ShoKubun", "Abst"]

# 課題番号の重複を確認。課題番号でソートする。
kadai["ID"].duplicated().any()
# kadai = kadai.set_index("ID") # IDをインデックスに設定するコード
kadai = kadai.sort_values("ID")
kadai.reset_index(inplace=True, drop=True)

# Abstの各項目が英語だけの場合、ブランクに入れ替える
# kadai.Abst1[~kadai["Title"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""
# kadai.Abst2[~kadai["Abst1"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""
# kadai.Abst3[~kadai["Abst2"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""
# kadai.Abst1[~kadai["Abst3"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""

# 研究開始時の研究の概要、研究成果の概要、研究実績の概要、キーワードを結合
# kadai['Abst'] = kadai['Title'].fillna('') + kadai['Abst1'].fillna('') + kadai['Abst2'].fillna('') + kadai['Abst3'].fillna('') + kadai['Keyword'] # t01
# kadai['Abst'] = kadai['Title'].fillna('') + '。' + kadai['Keyword'] + '。' + kadai['Abst1'].fillna('') + kadai['Abst2'].fillna('') + kadai['Abst3'].fillna('') # t02



# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadai))
print("概要が空白の課題数： %5d" % len(kadai[kadai["Abst"].isna()]))
kadai = kadai.dropna(subset=["Abst"])
print("空白を除いた課題数： %5d\n" % len(kadai))

# Abst中の改行コード、全角スペースを削除
kadai = kadai.replace('\r', '', regex=True) # Carriage Return(MacOS9) \r\n for Windows
kadai = kadai.replace('\n', '', regex=True) # Line Feed（Unix MacOSX）
# kadai = kadai.replace(' ', '', regex=True) # 半角スペースは英語があるので削除しない
kadai = kadai.replace('　', '', regex=True) # 全角スペース

# Abstが英語のみの課題を削除
num_jpen = len(kadai)
kadai = kadai[kadai["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadai)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d\n" % num_jp)

# 小区分が設定されていない課題を削除（旧分類、特設分野）
aaa = len(kadai)
kadai = kadai.dropna(subset=["ShoKubun"])
print("小区分がブランク： %5d" % (aaa - len(kadai)))
print("小区分の設定あり： %5d\n" % len(kadai))

# 小区分の文字列の数字部分だけを取り出す
kadai["ShoKubun"] = kadai["ShoKubun"].str[3:8]
kadai = kadai.astype({"ShoKubun": int})

kadai

### 整理した科研費データの保存
審査区分データを読み込み、小区分番号を参照して結合  
トレーニングデータと、テストデータに分けて保存する。

In [ ]:
from sklearn.model_selection import train_test_split

# 科研費の審査区分表データのcsvファイル
open_kubun_csv = "KubunTable.csv"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker

# 書き出し用CSVファイル名
train_csv = data_path + "kadai_train.csv"
test_csv  = data_path + "kadai_test.csv"

# ============================================================================

# 審査区分テーブルのロード
kubun_table = pd.read_csv(data_path + open_kubun_csv, encoding="cp932")
kubun_table = kubun_table[["tabDai", "tabSho"]]

# 審査区分表の重複を削除（一つの小区分が２つまたは３つの『中区分』に所属することに由来する）
print("重複削除前の項目数： %3d" % len(kubun_table))
kubun_table = kubun_table.drop_duplicates()
print("重複削除後の項目数： %3d\n" % len(kubun_table))

## 中区分への変換
## mergeを用いて、審査区分表のデータと突合
#print("統合前のデータ数： %5d" % len(kadai))
#kadaiChu = pd.merge(kadai, kubun_table, left_on='ShoKubun', right_on='tabSho')
#kadaiChu = kadaiChu[["Abst", "tabChu", "ID", "ShoKubun"]]
#print("統合したデータ数： %5d\n" % len(kadaiChu))

# 大区分への変換
# mergeを用いて、審査区分表のデータと突合
print("統合前のデータ数： %5d" % len(kadai))
kadaiDai = pd.merge(kadai, kubun_table, left_on='ShoKubun', right_on='tabSho')
kadaiDai = kadaiDai[["Abst", "tabDai", "ID", "ShoKubun"]]
print("統合したデータ数： %5d" % len(kadaiDai))

# 訓練用とテスト用に分割 層化
kadai_train, kadai_test =  train_test_split(kadaiDai, shuffle=True, stratify = kadaiDai["tabDai"].tolist())
print("トレーニングデータ数： %5d"   % len(kadai_train))
print("テストデータ数　　　： %5d\n" % len(kadai_test))



## 訓練用とテスト用に分割 層化
#kadai_train, kadai_test =  train_test_split(kadaiChu, shuffle=True, stratify = kadaiChu["tabChu"].tolist())
#print("トレーニングデータ数： %5d" % len(kadai_train))
#print("テストデータ数　　　： %5d\n" % len(kadai_test))

# 課題番号→小区分→中区分を基準にソート（計算的には不要だが、人間用にソートしておく）
kadai_train = kadai_train.sort_values(["tabDai", "ShoKubun", "ID"])
kadai_test  = kadai_test.sort_values (["tabDai", "ShoKubun", "ID"])

# ソート用に残していた課題番号（ID）行を削除
#kadai_train = kadai_train.drop(['ID'], axis=1)
#kadai_test  = kadai_test.drop (['ID'], axis=1)
kadai_train = kadai_train.drop(['ID', 'ShoKubun'], axis=1)
kadai_test  = kadai_test.drop (['ID', 'ShoKubun'], axis=1)

# csvとして書き出し
kadai_train.to_csv(train_csv, header=False, index=False)
kadai_test.to_csv (test_csv,  header=False, index=False)

print("Saved\n %s\n %s\n" % (train_csv, test_csv))

## ファインチューニングの実施
### モデルとTokenizerの読み込み
日本語の事前学習済みモデルと、これと紐づいたTokenizerを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=11) # 大区分は11
sc_model.cuda() # GPUを使う
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

print("Finish")

### データセットの読み込み
さきほど作成した科研費データ（training,test）を読み込みます。

In [ ]:
from datasets import load_dataset

def tokenize(batch):
    # return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)
    
data_path = "./"

train_data = load_dataset("csv", data_files=data_path+"news_train.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

test_data = load_dataset("csv", data_files=data_path+"news_test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

print("Finish")

## 評価用の関数
`sklearn.metrics`を使用し、モデルを評価するための関数を定義します。  


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

print("Finish")

### Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerの設定を行います。  
https://huggingface.co/transformers/main_classes/trainer.html  
https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments  

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ありません
    logging_dir = "./logs",
    save_total_limit=1,  # limit the total amount of checkpoints. Deletes the older checkpoints.
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data,
)

print("Finish")

### モデルの訓練
設定に基づきファインチューニングを行います。  
40分程度かかる。 colaboratory 32分かかった　sagemaker

In [ ]:
trainer.train()

print("Finish")

### モデルの評価
Trainerの`evaluate()`メソッドによりモデルを評価します。  
２分程度かかる

In [ ]:
trainer.evaluate()

print("Finish")

### TensorBoardによる結果の表示
TensorBoardを使って、logsフォルダに格納された学習過程を表示します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

### モデルの保存
訓練済みのモデルを保存します。

In [ ]:
# data_path = "/content/drive/My Drive/bert_nlp/section_5/"
data_path = "./"

sc_model.save_pretrained(data_path)
tokenizer.save_pretrained(data_path)

print("Finish")

# ここまでは不要

## 複数の学術文書を分類
学習済み（ファインチューニング済み）のモデルを読み込み、複数の学術文書（実際には科研費概要テキスト）を分類する

### モデルの読み込み
保存済みのモデルを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # SageMaker

loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
# loaded_model.cuda() # GPU対応
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

print("Finish")

### 分類精度検証用の科研費データ読み込み

In [ ]:
import pandas as pd

# 科研費データベースからダウンロードした未加工のcsvファイルを指定
# open_original_csv = "KibanC_2021_Original.csv"
open_original_csv = "KibanC_2022-2022.csv" # 直近の１年（2022年）
# open_original_csv = "KibanC_2019-2020.csv" # 2018は「研究開始時の研究の概要」が無い
#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "../data/"

# csvファイルを開く
raw_data2 = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()


raw_data2

In [ ]:
import pandas as pd

# 科研費データベースからダウンロードした未加工のcsvファイルを指定
# open_original_csv = "KibanC_2021_Original.csv"
open_original_csv = "KibanC_2022-2022.csv" # 直近の１年（2022年）
# open_original_csv = "KibanC_2019-2020.csv" # 2018は「研究開始時の研究の概要」が無い
#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "../data/"

# csvファイルを開く
raw_data2 = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()

# 今後必要な行だけを取り出し、リネーム
kadai2 = raw_data2[["研究課題/領域番号", "審査区分", "研究開始時の研究の概要"]]
kadai2.columns = ["ID", "ShoKubun", "Abst"]

# 課題番号の重複を確認。課題番号でソートする。
kadai2["ID"].duplicated().any()
# kadai2 = kadai2.set_index("ID") # IDをインデックスに設定するコード
kadai2 = kadai2.sort_values("ID")

# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadai2))
print("概要が空白の課題数： %5d" % len(kadai2[kadai2["Abst"].isna()]))
kadai2 = kadai2.dropna(subset=["Abst"])
print("空白を除いた課題数： %5d" % len(kadai2))

# Abst中の改行コードを削除
kadai2 = kadai2.replace('\r', '', regex=True)
kadai2 = kadai2.replace('\n', '', regex=True)

# Abstが英語のみの課題を削除
num_jpen = len(kadai2)
kadai2 = kadai2[kadai2["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadai2)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d" % num_jp)

# kadai.to_csv(data_path + "test1.csv", encoding = "cp932")

# 小区分が設定されていない課題を削除（旧分類、特設分野）
aaa = len(kadai2)
kadai2 = kadai2.dropna(subset=["ShoKubun"])
print("小区分がブランク： %5d" % (aaa - len(kadai2)))
print("小区分の設定あり： %5d" % len(kadai2))

# 小区分の文字列の数字部分だけを取り出す
kadai2["ShoKubun"] = kadai2["ShoKubun"].str[3:8]
kadai2 = kadai2.astype({"ShoKubun": int})

print("Finish")

### 大区分に変換

In [ ]:
#import pandas as pd
#from sklearn.model_selection import train_test_split

# 科研費の審査区分表データのcsvファイル
open_kubun_csv = "KubunTable.csv"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagamaker

# 審査区分テーブルのロード
kubun_table = pd.read_csv(data_path + open_kubun_csv, encoding="cp932")
kubun_table = kubun_table[["tabDai", "tabSho"]]

# 審査区分表の重複を削除（一つの小区分が２つまたは３つの『中区分』に所属することに由来する）
print("審査区分表の整理")
print("重複削除前の項目数： %3d" % len(kubun_table))
kubun_table = kubun_table.drop_duplicates()
print("重複削除後の項目数： %3d" % len(kubun_table))

# 大区分への変換
# mergeを用いて、審査区分表のデータと突合
print("統合前のデータ数： %5d" % len(kadai2))
kadaiDai2 = pd.merge(kadai2, kubun_table, left_on='ShoKubun', right_on='tabSho')
kadaiDai2 = kadaiDai2[["Abst", "tabDai", "ID", "ShoKubun"]]
print("統合したデータ数： %5d" % len(kadaiDai2))

kadaiDai2.info()

show_num = 0
kadaiDai2["Abst"][show_num]

print("Finish")

### 分類精度を複数ファイルで確認
３時間程度かかるかも  
sagemakerでは40分程度

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

results_binary = 'results'
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]


max_length = 512

num_data = len(kadaiDai2)
print("分類する課題数： %d" % num_data)
# num_data = 100
num_category = len(Daikubun)
# results = torch.zeros(num_data, num_category) # テンソルで変数を用意
results = np.zeros((num_data, num_category)) # メモリーが足りないと言われるのでテンソルではなくnumpy arrayにしてみた

for m in tqdm(range(num_data)):
  words = loaded_tokenizer.tokenize(kadaiDai2["Abst"][m])
  word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
  word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換
  
  y = loaded_model(word_tensor) # GPU未対応時の予測
  # y = loaded_model(word_tensor.cuda())  # GPU対応時の予測

  # results[m,:] = y[0]
  results[m,:] = y[0].detach().numpy() # テンソルをnumpy arrayに変換

# 変数をとりあえずバイナリで保存
np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存

print(results.shape)
results

### ここで一度保存
時間がかかる処理のあとなので、この段階で生成できたデータをファイルに保存

In [ ]:
# コードが不十分
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

results = np.load(data_path+results_binary+".npy")

# 変数をとりあえずバイナリで保存
#np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存
np.savez(data_path+"categolization_results", results, kadaiDai2, Daikubun)

分類精度を表示
混同行列

## 分類結果を表示

### 分類精度を複数ファイルで確認
confusion matrix （混同行列）を作成  
「マルチラベリング」で対応する方法もありそう

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

show_num = 0

results_binary = "results"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

# num_category = len(Daikubun)

results_2 = np.load(data_path+results_binary+".npy")

#results_3 = np.load(data_path+"categolization_results.npz", allow_pickle=True)
#results_2 = results_3['arr_0']
#kadaiDai2 = results_3['arr_1']
#Daikubun  = results_3['arr_2']

print("ロードした推定確率データのサイズ %d %d" % results_2.shape)

num_category = len(Daikubun)

results_2 = torch.tensor(results_2) # Softmax関数を使うためにテンソルに変換
m = torch.nn.Softmax(dim=1) # Softmax関数で確率に変換
results_2 = m(results_2)
results_2 = results_2.numpy() # numpy arrayに戻す

kadaiDai2['estimated'] = np.argmax(results_2, axis=1) # 最大要素のindexを返す（確率が最大の区分を返す）


print("重複課題の削除前 %5d" % len(kadaiDai2))
kadaiDai3 = kadaiDai2.drop_duplicates(subset='ID', keep=False)
print("重複課題の削除後 %5d" % len(kadaiDai3))

duplicated_data = kadaiDai3[kadaiDai3.duplicated(subset='ID', keep=False)]
unique_id = duplicated_data['ID'].drop_duplicates()


#
#for kk in unique_id:
#  dup_set = duplicated_data[duplicated_data['ID'] == kk]
#  cat_est = dup_set['estimated'].tolist()[0]
#  cat_real = dup_set['tabDai'].tolist()
#
#  if cat_est in cat_real:
#    aaa = cat_real.index(cat_est)
#    print(dup_set[:aaa])
#  else:
#    aaa = cat_est
#
#print(aaa)
    





# https://analysis-navi.com/?p=553
confmat = confusion_matrix(kadaiDai3['tabDai'], kadaiDai3['estimated']) # 混同行列の取得。(true, predicted)の順番
confmat = pd.DataFrame(confmat,columns=["pred_" + str(l) for l in Daikubun], index=["act_" + str(l) for l in Daikubun])
# print(cm)


#dup_idx_all = kadaiDai2.drop_duplicated(subset='ID',keep=False)

#duplicated_data_all = kadaiDai2[dup_idx_all]
#duplicated_data_all = duplicated_data_all[['ID', 'tabDai', 'estimated']]
#print(duplicated_data_all.shape)
#duplicated_data_all = duplicated_data_all.groupby('ID').count()

#dup_idx_first = kadaiDai2[kadaiDai2.duplicated(subset='ID', keep='first')]
#dup_idx_first = dup_idx_first.index

#for kk in dup_idx_first:
#  print(kk)


#duplicated_data.to_csv(data_path+"duplicated_mat.csv")

#kubun_estimated = np.argmax(results_2, axis=1)
#kubun_real      = kadaiDai2["tabDai"]
#
#kubun_results = pd.DataFrame(
#    data={
#        'estimated':kubun_estimated,
#        'real':kubun_real
#    }
#)

# 結果のグラフを表示
plt.bar(Daikubun, results_2[show_num,:])
print(kadaiDai3["Abst"][show_num])
print(kadaiDai3["ShoKubun"][show_num])
print(kadaiDai3['ID'][show_num])
print("実際　大区分" + Daikubun[kadaiDai3["tabDai"][show_num]])
print("推定　大区分" + Daikubun[kadaiDai3["estimated"][show_num]])
print(int(results_2[show_num, kadaiDai3["estimated"][show_num]]*100))
#kadaiDai2 = kadaiDai2[["Abst", "tabDai", "ID", "ShoKubun"]]

#out_mat = kubun_results.value_counts(sort=False)
#kubun_results.aggregate


#out_mat.to_csv(data_path+"result_mat.csv")

#dup_idx_first
confmat

# 以下不要

### 学術分野の分類
読み込んだモデルを使って学術分野を分類します。

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch

import matplotlib.pyplot as plt
import numpy as np

Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

# #@title String fields
# sample_text = '' #@param {type:"string"}

# sample_text = "磁性ナノ粒子を用いた新しい診断治療技術に関する研究課題である。腫瘍等に集積させた磁性ナノ粒子に体外から比較的低い周波数の交流磁界を印加し、そのときに生じる磁気信号を検出することにより体内の画像診断が可能となる。また、より高い周波数の交流磁界を印加すると磁性ナノ粒子が発熱する。この発熱は癌の温熱治療（ハイパーサーミア）に利用することができる。交流磁界に対する磁性ナノ粒子の磁化応答（ダイナミクス）を解明し、これら診断治療の実用を目指す。" # 1
# sample_text = "国際的な歴史的文字の共通検索実現およびオープンデータ化を目指して、奈文研・東京大学史料編纂所・国文学研究資料館・国立国語研究所・台湾中央研究院歴史語言研究所を中心に、京都大学人文学研究所や中国社会科学院歴史研究所等の研究者の参加も得ながら、国内・国外で各1回、研究会を開催した。そして、歴史的文字画像データ共通検索の基本コンセプトを、「開かれた」「対等」「継続的」として、このコンセプト基づいた「共通検索のためのるフレームワークの構築」作業に着手し、具体的な取り決めを決定した。この内容については、朝日新聞等でも報道された。詳細は『奈良文化財研究所紀要2019』に掲載予定である。また、研究の促進と参加誘発を目指して、奈良文化財研究所が公開している木簡関連の総合データベース「木簡庫」に検索した木簡情報（釈文・メタデータ等）をCSVファイルでダウンロードすることができる機能を追加し、オープンデータとした。これにより、利用者の目的に応じた木簡データのダウンロードおよびそれを活用しての研究や、さらにはそれぞれの関心に基づくデータベースの作成・公開も可能になった。この内容についても、朝日新聞・NHK等で報道された。文字に関する知識の集積作業として、あらたに気づきメモ約13000文字分（のべ）、観察記録シート約15000文字分（のべ）の情報を収集した。従来からの蓄積と合わせると、観察記録シートによる文字情報は木簡庫で公開している文字画像の約25％（テキストで公表している釈文文字数の約10％）に到達した。文字画像の切り出し・公開は1067字分行った。また、観察記録シートの手法を試験的に用い、中国晋代簡牘・韓国新羅木簡・日本平城宮木簡の比較を行い、それぞれの親和性を検討した研究を行い、成果を得た。この他、木簡の調査者との情報共有のためのワークショップを開催した。" # 0
# sample_text = "本研究は、最新科学と考古学の有機的なコラボレーションによって、発掘によらない王陵級巨大古墳の調査研究の方法を確立することにある。巨大古墳は、発掘調査の禁止あるいは制限がかかっている。そのため、重要な歴史資料でありながら、内容が不明なことが多い。そうした現状を最新科学を用いて打破する。具体的にはミュオンという素粒子を用いてレントゲン写真のように古墳内部を透視する。さらに出土埴輪の形態学的分析＋化学分析・墳丘のレーザー測量を行う。主たるフィールドを吉備に置き、その調査成果を畿内の王陵と比較し、王陵級巨大古墳の構造分析を行う。" # 0
# sample_text = "本研究は、低エネルギー動作を特徴とする断熱的量子磁束回路(AQFP)を用いた双方向演算が可能な可逆回路の学理を明らかにし、論理回路の熱力学的極限を超える究極の低消費エネルギー集積回路を実現する。これにより回路の消費エネルギーを半導体回路に対して６桁以上低減し、冷却電力を考慮しても十分な優位性を生み出す。本研究は可逆AQFP を中核技術とし、回路設計技術、新規可逆回路、プロセッサアーキテクチャ、磁性体を用いた位相シフトAQFP、3 次元集積回路技術を研究し、超省エネ集積回路の基盤技術を確立する。最終目標として100nW 以下の動作が可能な4b可逆AQFPプロセッサの実現を目指す。" # 1
# sample_text = "ストリゴラクトンは根から分泌されて土壌中でAM菌との共生を促進する根圏シグナル物質である。AM菌共生は植物の陸上進出を可能にし、さらに陸上でのその後の繁栄を支えてきた。種子植物はSL受容体をもっており、SLは個体内で成長を調節する植物ホルモンとしても働き、養分吸収と成長のバランスを制御して植物の成長を最適化する。本研究では、植物がAM菌との共生関係を構築し、それに合わせて成長を調節する仕組みを進化させた道筋を分子レベルで理解することをめざす。本研究により、地球が緑の惑星となりえた理由の一端を明らかにすることができる。" # 2
# sample_text = "発芽は、植物の一生において最も重要なイベントの1つである。様々な環境で初期生育を達成するために、植物は最適なタイミングで発芽する機構を進化させてきた。適切な発芽管理は農業的にも重要であり、特に種子を収穫する穀物植物においては直接収量に関わる。本研究では最近確立されたGWASシステムを用いて、世界的に栽培されている穀物植物であるイネにおいて温度依存的な発芽調節機構を明らかにすることを目的に研究を行う。"
# sample_text = "食品中のトランス脂肪酸は、過剰摂取により心疾患のリスクを高めることが疫学調査によって報告され、世界中で注目を集めている。しかしながら、食事によって摂取されたトランス脂肪酸の体内動態は明らかにされておらず、心疾患とトランス脂肪酸の直接的な関連性は不明である。そこで本研究では、心疾患とトランス脂肪酸の因果関係の解明に資する基礎的なデータを蓄積することを目的とし、食品中に多く含まれる炭素数18、二重結合数1の13種類のトランス脂肪酸（trans-18:1）異性体をマウスに投与し、各臓器・組織中のtrans-18:1異性体を各種質量分析計で測定することで、トランス脂肪酸異性体の体内動態を明らかにする。"
# sample_text = "本研究では、遅延時間が小さく、100％の回線利用率を達成する理想的なインターネット輻輳（ふくそう）制御手法を実現することを目的とする。具体的には、バッファブロートと呼ばれる、遅延時間が数秒クラスに増大する輻輳問題に着目する。従来のインターネット輻輳制御手法では用いられていないネットワーク計測手法を活用し、数学的理論に基づいて輻輳状態を正確に推定する。その結果を、新たな輻輳制御手法へ応用する。提案手法の有効性を数学的に保証すると共に、コンピュータシミュレーション及び実ネットワーク環境下での実験により実用性の検証を行う。"
# sample_text = "現代の倫理学では、カントの倫理学は「義務論」に分類され、アリストテレス流の「徳倫理学」やベンサムに代表される「功利主義」と対比されるのが一般的である。しかしカント自身は晩年の『道徳の形而上学』において、義務論の枠内で徳論を展開している。本研究は、こうしたカントの徳倫理学の独自性を思想史的に遡って解明するとともに、それが倫理学の理論としてもつ強みについても明らかにする。"

# Abst中の改行コードを削除
sample_text = sample_text.replace('\r', '')
sample_text = sample_text.replace('\n', '')
print(sample_text)


max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
print(len(word_ids))
word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換


x = word_tensor.cuda()  # GPU対応
# x = word_tensor # GPU未対応==============================================
y = loaded_model(x)  # 予測
y = y[0]
pred = y.argmax(-1)  # 最大値のインデックス
out_put = Daikubun[pred]
print("大区分"+out_put)

m = torch.nn.Softmax(dim=1) # Softmax関数で確率に変換
y = m(y)
yy = y.tolist()[0]
yy = list(map(lambda x: int(x*100), yy))
all_result = dict(zip(Daikubun, yy))
print(all_result)

# 結果のグラフを表示
plt.bar(Daikubun, yy)